In [1]:
open System
open System.IO
open System.Text.RegularExpressions

In [2]:
#r "nuget: FParsec"
open FParsec

Installed Packages FParsec, 1.1.1

In [3]:
type Mapping = 
    {Destination: uint; Source: uint; Length: uint}
    with 
    static member create x y z = {Destination = x; Source = y; Length = z}
    static member tryMap (value:uint) {Destination = d ;Source = s; Length = len}  = 
        if value >= s && value <= s + len - 1u then
            Some ( d + (value - s) ) 
        else 
            None
    member this.End = this.Source + this.Length - 1u
    static member map (value:uint) {Destination = d ;Source = s; Length = len} = 
        d + (value - s)
type ThingMap = 
    {From: string; To: string; Map: Mapping list}
    with
    static member create from _to mappings = {From = from; To = _to; Map = mappings}
    member self.Item 
        with get(index:uint) : uint = 
            self.Map 
            |> List.tryPick (Mapping.tryMap index)
            |> Option.defaultValue index


In [4]:
let test = 
    """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [5]:
let lineToMapping (line: string):Mapping = 
    let p = spaces >>. puint32 .>> spaces 
    match run (pipe3 p p p Mapping.create) line with 
    | Success(result, _,_) -> result 
    | _ -> failwith $"Problem with {line}"


In [6]:
let parseBlock (block: string) = 
    let lines = block.Split("\n")
    let [from;_to] = 
        Regex.Match(lines[0], @"(\w+)-to-(\w+)").Groups
        |> Seq.tail 
        |> Seq.map( fun x -> x.Value)
        |> Seq.toList
    let numbers = 
        Array.tail lines
        |> Array.map lineToMapping
        |> List.ofArray
    ThingMap.create from _to numbers


In [7]:
let testseeds = [for m in Regex.Matches(Regex(@"\n\n").Split(test)[0], @"\d+") -> System.UInt32.Parse m.Value]
let testblocks =
    Regex(@"\n\n").Split(test) 
    |> Array.tail
    |> Array.map parseBlock
    |> List.ofArray

In [8]:
let rec getLocation (blocks: ThingMap list) (current:uint) = 
    match blocks with 
    | [] -> current 
    | block::rest -> getLocation rest (block[current])


In [9]:
testseeds 
|> List.map (getLocation testblocks)
|> List.min

35

In [10]:
let input = File.ReadAllText("input_5.txt")
let seeds = [for m in Regex.Matches(Regex(@"\n\n").Split(input)[0], @"\d+") -> System.UInt32.Parse m.Value]
let blocks =
    Regex(@"\n\n").Split(input)
    |> Array.tail
    |> Array.map parseBlock
    |> List.ofArray

In [11]:
seeds 
|> List.map (getLocation blocks)
|> List.min

88151870

# Part2

In [55]:
type Range = {Start: uint; End: uint; mutable Mapped: bool}
let testRanges = 
    testseeds 
    |> List.chunkBySize 2
    |> List.map (fun [x;y] -> {Start = x; End = x + y - 1u; Mapped = false})
    

In [56]:
testblocks[0].Map[1]

Destination,52
Source,50
Length,48
End,97


In [123]:
let next (mapping: Mapping) (range: Range) =
    let mapRange r = 
        {r with Start = Mapping.map r.Start mapping; End = Mapping.map r.End mapping}
    if range.Mapped then 
        [range] 
    else 
        [
            if range.End < mapping.Source || mapping.End < range.Start then 
                // No overlap
                range
            else 
                if range.Start < mapping.Source then 
                    {Start = range.Start; End = mapping.Source - 1u; Mapped = false}
                    if range.End <= mapping.End then 
                        {Start = mapping.Source; End = range.End; Mapped = true}
                    else 
                        {Start = mapping.Source; End = mapping.End; Mapped = true}
                        {Start = mapping.End + 1u; End = range.End; Mapped = false}
                else 
                    if range.End <= mapping.End then 
                        {Start = range.Start; End = range.End; Mapped = true}
                    else
                        {Start = range.Start; End= mapping.End; Mapped = true}
                        {Start = range.End + 1u; End = range.End; Mapped = false}
        ]
        |> List.map (fun x -> 
            if x.Mapped then mapRange x else x)

In [180]:
let next2 (mapping: Mapping) (range: Range) =
    let mapRange r = 
        {r with Start = Mapping.map r.Start mapping; End = Mapping.map r.End mapping}
    if range.Mapped then 
        [range] 
    else if range.End < mapping.Source || mapping.End < range.Start then 
        [range]
    else 
        [
            if mapping.Source>0u then 
                {Start = range.Start; End = min range.End (mapping.Source - 1u); Mapped = false}
            {Start = max range.Start mapping.Source; End = min range.End mapping.End; Mapped = true}
            if mapping.End < System.UInt32.MaxValue then
                {Start = max range.Start (mapping.End + 1u); End = range.End; Mapped = false}
        ]
        |> List.filter(fun x -> x.End>= x.Start)
        |> List.map (fun x -> 
            if x.Mapped then mapRange x else x)

In [169]:
let test (mapping: Range) (range: Range) =
    if range.Mapped then 
        [range] 
    else if range.End < mapping.Start || mapping.End < range.Start then 
        [range]
    else 
        [
            if mapping.Start>0u then 
                {Start = range.Start; End = min range.End (mapping.Start - 1u); Mapped = false}
            {Start = max range.Start mapping.Start; End = min range.End mapping.End; Mapped = true}
            if mapping.End < System.UInt32.MaxValue then
                {Start = max range.Start (mapping.End + 1u); End = range.End; Mapped = false}
        ]
        |> List.filter(fun x -> x.End>= x.Start)

In [167]:
0u - 1u

4294967295

In [178]:
test {Start=0u;End=4u;Mapped = false}  {Start=1u; End=3u;Mapped = false} 
|> Array.ofList

index value 0 { Start = 1u\n End = 3u\n Mapped = true } Start 1 End 3 Mapped True Mapped@ True

In [141]:
let next3 (mapping: Mapping) (range: Range) =
    let mapRange r = 
        {r with Start = Mapping.map r.Start mapping; End = Mapping.map r.End mapping}
    if range.Mapped then 
        [range] 
    else 
        let a,b = range.Start, range.End 
        let x,y = mapping.Source, mapping.End
        [a; b; x; min b (x - 1u); max a b; min b y; max a (y + 1u)]
        |> List.sort
        |> List.windowed 2
        |> List.map (fun [x;y] -> {Start = x; End = y; Mapped = false})
        |> List.filter (fun r -> r.End >= r.Start)
        |> List.filter (fun r -> r.Start >= a && r.End<=b)
        |> List.map (fun r -> if r.Start >= x && r.End <= y then {mapRange r with Mapped=true} else r )

In [124]:
let range = testRanges[0]
let mapping = testblocks[0].Map[1]
next mapping range

[ { Start = 81u
 End = 94u
 Mapped = true } ] HeadOrDefault { Start = 81u\n End = 94u\n Mapped = true } Start 81 End 94 Mapped True Mapped@ True TailOrNull [ ] HeadOrDefault <null> TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4136\r\n at lambda_method422(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663766 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\Private\.nuget\packages\microsoft.dotnet-interactive\1.0.456201\tools\net7.0\any\FSharp.Core.dll ModuleVersionId 6dbe05a8-5c53-690c-c522-0b7a960bbd24 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=7.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a CodeBase file:///C:/Users/Private/.nuget/packages/microsoft.dotnet-interactive/1.0.456201/tools/net7.0/any/FSharp.Core.dll FullName FSharp.Core, Version=7.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a EntryPoint <null> DefinedTypes [ Microsoft.FSharp.Core.Unit, Microsoft.FSharp.Core.SourceConstructFlags, Microsoft.FSharp.Core.CompilationRepresentationFlags, Microsoft.FSharp.Core.SealedAttribute, Microsoft.FSharp.Core.AbstractClassAttribute, Microsoft.FSharp.Core.EqualityConditionalOnAttribute, Microsoft.FSharp.Core.ComparisonConditionalOnAttribute, Microsoft.FSharp.Core.AllowNullLiteralAttribute, Microsoft.FSharp.Core.VolatileFieldAttribute, Microsoft.FSharp.Core.DefaultAugmentationAttribute, Microsoft.FSharp.Core.CLIEventAttribute, Microsoft.FSharp.Core.CLIMutableAttribute, Microsoft.FSharp.Core.AutoSerializableAttribute, Microsoft.FSharp.Core.DefaultValueAttribute, Microsoft.FSharp.Core.EntryPointAttribute, Microsoft.FSharp.Core.ReferenceEqualityAttribute, Microsoft.FSharp.Core.StructuralComparisonAttribute, Microsoft.FSharp.Core.StructuralEqualityAttribute, Microsoft.FSharp.Core.NoEqualityAttribute, Microsoft.FSharp.Core.CustomEqualityAttribute ... (2190 more) ] IsCollectible False ManifestModule FSharp.Core.dll ReflectionOnly False Location C:\Users\Private\.nuget\packages\microsoft.dotnet-interactive\1.0.456201\tools\net7.0\any\FSharp.Core.dll ImageRuntimeVersion v4.0.30319 GlobalAssemblyCache False HostContext 0 IsDynamic False ExportedTypes [ Microsoft.FSharp.Core.Unit, Microsoft.FSharp.Core.SourceConstructFlags, Microsoft.FSharp.Core.CompilationRepresentationFlags, Microsoft.FSharp.Core.SealedAttribute, Microsoft.FSharp.Core.AbstractClassAttribute, Microsoft.FSharp.Core.EqualityConditionalOnAttribute, Microsoft.FSharp.Core.ComparisonConditionalOnAttribute, Microsoft.FSharp.Core.AllowNullLiteralAttribute, Microsoft.FSharp.Core.VolatileFieldAttribute, Microsoft.FSharp.Core.DefaultAugmentationAttribute, Microsoft.FSharp.Core.CLIEventAttribute, Microsoft.FSharp.Core.CLIMutableAttribute, Microsoft.FSharp.Core.AutoSerializableAttribute, Microsoft.FSharp.Core.DefaultValueAttribute, Microsoft.FSharp.Core.EntryPointAttribute, Microsoft.FSharp.Core.ReferenceEqualityAttribute, Microsoft.FSharp.Core.StructuralComparisonAttribute, Microsoft.FSharp.Core.StructuralEqualityAttribute, Microsoft.FSharp.Core.NoEqualityAttribute, Microsoft.FSharp.Core.CustomEqualityAttribute ... (214 more) ] IsFullyTrusted True CustomAttributes [ [Microsoft.FSharp.Core.FSharpInterfaceDataVersionAttribute((Int32)2, (Int32)0, (Int32)0)], [System.Runtime.Versioning.TargetFrameworkAttribute(".NETStandard,Version=v2.1", FrameworkDisplayName = ".NET Standard 2.1")], [System.Reflection.AssemblyCompanyAttribute("Microsoft Corporation")], [System.Reflection.AssemblyConfigurationAttribute("Release")], [System.Reflection.AssemblyCopyrightAttribute("© Microsoft Corporation. All rights reserved.")], [System.

In [125]:
let mapThroughBlock next (block: ThingMap) (range: Range) = 
    let rec mapThroughLayers (mappings : Mapping list) (ranges: Range list) =
        match mappings with 
        | [] -> ranges 
        | mapping::rest -> 
            let newRanges =
                ranges 
                |> List.map (next mapping) 
                |> List.concat
            mapThroughLayers rest newRanges
    mapThroughLayers block.Map [range]

In [171]:
mapThroughBlock next2 (testblocks[0]) (testRanges[0])

[ { Start = 81u
 End = 94u
 Mapped = true } ] HeadOrDefault { Start = 81u\n End = 94u\n Mapped = true } Start 81 End 94 Mapped True Mapped@ True TailOrNull [ ] HeadOrDefault <null> TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4136\r\n at lambda_method422(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetVa... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663766 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\Private\.nuget\packages\microsoft.dotnet-interactive\1.0.456201\tools\net7.0\any\FSharp.Core.dll ModuleVersionId 6dbe05a8-5c53-690c-c522-0b7a960bbd24 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=7.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a CodeBase file:///C:/Users/Private/.nuget/packages/microsoft.dotnet-interactive/1.0.456201/tools/net7.0/any/FSharp.Core.dll FullName FSharp.Core, Version=7.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a EntryPoint <null> DefinedTypes [ Microsoft.FSharp.Core.Unit, Microsoft.FSharp.Core.SourceConstructFlags, Microsoft.FSharp.Core.CompilationRepresentationFlags, Microsoft.FSharp.Core.SealedAttribute, Microsoft.FSharp.Core.AbstractClassAttribute, Microsoft.FSharp.Core.EqualityConditionalOnAttribute, Microsoft.FSharp.Core.ComparisonConditionalOnAttribute, Microsoft.FSharp.Core.AllowNullLiteralAttribute, Microsoft.FSharp.Core.VolatileFieldAttribute, Microsoft.FSharp.Core.DefaultAugmentationAttribute, Microsoft.FSharp.Core.CLIEventAttribute, Microsoft.FSharp.Core.CLIMutableAttribute, Microsoft.FSharp.Core.AutoSerializableAttribute, Microsoft.FSharp.Core.DefaultValueAttribute, Microsoft.FSharp.Core.EntryPointAttribute, Microsoft.FSharp.Core.ReferenceEqualityAttribute, Microsoft.FSharp.Core.StructuralComparisonAttribute, Microsoft.FSharp.Core.StructuralEqualityAttribute, Microsoft.FSharp.Core.NoEqualityAttribute, Microsoft.FSharp.Core.CustomEqualityAttribute ... (2190 more) ] IsCollectible False ManifestModule FSharp.Core.dll ReflectionOnly False Location C:\Users\Private\.nuget\packages\microsoft.dotnet-interactive\1.0.456201\tools\net7.0\any\FSharp.Core.dll ImageRuntimeVersion v4.0.30319 GlobalAssemblyCache False HostContext 0 IsDynamic False ExportedTypes [ Microsoft.FSharp.Core.Unit, Microsoft.FSharp.Core.SourceConstructFlags, Microsoft.FSharp.Core.CompilationRepresentationFlags, Microsoft.FSharp.Core.SealedAttribute, Microsoft.FSharp.Core.AbstractClassAttribute, Microsoft.FSharp.Core.EqualityConditionalOnAttribute, Microsoft.FSharp.Core.ComparisonConditionalOnAttribute, Microsoft.FSharp.Core.AllowNullLiteralAttribute, Microsoft.FSharp.Core.VolatileFieldAttribute, Microsoft.FSharp.Core.DefaultAugmentationAttribute, Microsoft.FSharp.Core.CLIEventAttribute, Microsoft.FSharp.Core.CLIMutableAttribute, Microsoft.FSharp.Core.AutoSerializableAttribute, Microsoft.FSharp.Core.DefaultValueAttribute, Microsoft.FSharp.Core.EntryPointAttribute, Microsoft.FSharp.Core.ReferenceEqualityAttribute, Microsoft.FSharp.Core.StructuralComparisonAttribute, Microsoft.FSharp.Core.StructuralEqualityAttribute, Microsoft.FSharp.Core.NoEqualityAttribute, Microsoft.FSharp.Core.CustomEqualityAttribute ... (214 more) ] IsFullyTrusted True CustomAttributes [ [Microsoft.FSharp.Core.FSharpInterfaceDataVersionAttribute((Int32)2, (Int32)0, (Int32)0)], [System.Runtime.Versioning.TargetFrameworkAttribute(".NETStandard,Version=v2.1", FrameworkDisplayName = ".NET Standard 2.1")], [System.Reflection.AssemblyCompanyAttribute("Microsoft Corporation")], [System.Reflection.AssemblyConfigurationAttribute("Release")], [System.Reflection.AssemblyCopyrightAttribute("© Microsoft Corporation. All rights reserved.")], [System.

In [182]:
let rec getMinLocation next (blocks: ThingMap list) (ranges: Range list) = 
    match blocks with 
    | [] -> ranges 
    | block::rest -> 
        ranges 
        |> List.map (mapThroughBlock next block) 
        |> List.concat
        |> List.map (fun x -> x.Mapped <- false; x)
        |> getMinLocation next rest


In [133]:
let logger range = printfn $"{range}"; range

In [ ]:
getMinLocation next testblocks testRanges
|> List.map (fun x -> x.Start) 
|> List.min

In [183]:
getMinLocation next2 testblocks testRanges
|> List.map (fun x -> x.Start) 
|> List.min

46

In [187]:
getMinLocation next2 blocks ranges
|> List.map (fun x -> x.Start) 
|> List.min

2008785